In [1]:
import sys

sys.path.insert(0, 'c:\\Users\\marco\\OneDrive\\Desktop\\Fisica\\ABM-Project\\Progetto\\MAS\\Lux\\model')

In [2]:
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels
from conf import RESULT_DIR, TESTNUM
from scipy.stats import kstest, norm, kurtosis
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox


In [3]:
plt.style.use('ggplot')

In [4]:
with open(f'{RESULT_DIR}/conf.pkl', 'rb') as f:
    CONST = pickle.load(f)

if not os.path.exists(RESULT_DIR + '/immagini'):
    os.mkdir(f'{RESULT_DIR}/immagini')

In [5]:
RESULT_DIR #= 'results/test_8/'

'results/notsignedpricevariation_02'

In [6]:
df_list = list([pd.read_pickle(f"{RESULT_DIR}/Model_vars_{i}.pkl") for i in range(CONST['N_RUNS'])])

In [7]:
pvalues = [adfuller(list(df_list[i].loc[:,'price']))[1] for i in range(len(df_list))]
volclus = [acorr_ljungbox(list(df_list[i].loc[:,'price']), lags = [20], return_df = True) for i in range(len(df_list))]
volclusvalues = [volclus[i]['lb_pvalue'].values[0]  for i in range(len(volclus))]
kurtosisvalues = [kurtosis(pd.DataFrame(df_list[i]['price'].values[1:] - df_list[i]['price'].values[:-1]).rolling(20, step = 2).sum()[10:]) for i in range(len(df_list))]

In [113]:
for i, model_df in enumerate(df_list):
    fig , ax = plt.subplots(3,1, figsize=(20, 16),dpi=300)
    ax[0].set_xlabel(f'Price time series - run {i}')
    ax[1].set_xlabel(f'Technical fraction run {i}')
    ax[2].set_xlabel(f'Opinion Index run {i}')
    ax[0].plot(model_df.price, label=f'Run {i}')
    ax[1].plot(model_df.technical_fraction, label=f'Run {i}')
    ax[2].plot(model_df.opinion_index, label = f'Run {i}')
    # fig.tight_layout(pad=5.0)
    title = ['DF-Test pvalue = ', round(pvalues[i],3), ' \u21FE ','Unit Root:', pvalues[i]>0.05,'\n'
             'VC-Test pvalue = ', round(volclusvalues[i],3), ' \u21FE ','Volatility Clustering:', volclusvalues[i]<0.05,'\n' 
             'Kurtosis value = ', kurtosisvalues[i].round(3),' \u21FE ','Leptokurtic:', kurtosisvalues[i]>0,'\n'    ]
    title = ' '.join(str(e) for e in title)
    fig.suptitle(title,fontsize=22,color='black')
    fig.savefig(f'{RESULT_DIR}/immagini/Price and technical fraction over time run={i}',facecolor='white', transparent=False)
    plt.close(fig)